In [103]:
import os
# import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
import pandas as pd
import numpy as np
import json
import time
# from config.settings.base import CAFE, DATA_DIR, CSV_DATA_PATH
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from tqdm.notebook import tqdm
import re
from tqdm import *
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [104]:
chromedriver = '/Users/dc/practice/recommendation/recomm_drf/cafe/chromedriver'
#chromedriver = '/Users/dc/study_dir/recommendation/cafe/chromedriver'
driver = webdriver.Chrome(chromedriver) 

/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_90941/4251084422.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


In [105]:
df = pd.read_csv(os.path.join('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe', '성동구_카페 정보.csv'), sep=',')

In [93]:
#df = pd.read_csv(os.path.join('/Users/dc/study_dir/recommendation/data/csv/cafe', '종로구_카페 정보.csv'), sep=',')

In [94]:
# df['dong'] = df.iloc[:, 3].str.split(' ').str[1]
# print(df['dong'])

In [95]:
# df.head(10)

In [106]:
df['place_name']

0          블루보틀 성수 카페
1              할아버지공장
2                  쎈느
3              어니언컴퍼니
4                대림창고
            ...      
670              브라운팟
671            플디 성수점
672         카페노티드 피치스
673              천상가옥
674    비엔나커피하우스 서울숲역점
Name: place_name, Length: 675, dtype: object

In [107]:
df['naver_keyword'] = df['place_name']
df['naver_map_url'] = ''
df.head(10)

,Unnamed: 0,id,place_name,address_name,road_address_name,phone,category_group_name,category_name,place_url,naver_keyword,naver_map_url
0,0,1492599844,블루보틀 성수 카페,서울 성동구 성수동1가 656-302,서울 성동구 아차산로 7,1533-6906,카페,음식점 > 카페,http://place.map.kakao.com/1492599844,블루보틀 성수 카페,
1,1,1301973155,할아버지공장,서울 성동구 성수동2가 309-133,서울 성동구 성수이로7가길 9,010-3934-2301,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/1301973155,할아버지공장,
2,2,1783691580,쎈느,서울 성동구 성수동2가 314-2,서울 성동구 연무장5길 20,02-469-2122,카페,음식점 > 카페,http://place.map.kakao.com/1783691580,쎈느,
3,3,145791269,어니언컴퍼니,서울 성동구 성수동2가 277-135,서울 성동구 아차산로9길 8,02-1644-1941,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/145791269,어니언컴퍼니,
4,4,1128989913,대림창고,서울 성동구 성수동2가 322-32,서울 성동구 성수이로 78,02-499-9669,카페,음식점 > 카페 > 테마카페 > 갤러리카페,http://place.map.kakao.com/1128989913,대림창고,
5,5,286483883,센터커피 서울숲점,서울 성동구 성수동1가 685-478,서울 성동구 서울숲2길 28-11,070-8868-2008,카페,음식점 > 카페 > 커피전문점,http://place.map.kakao.com/286483883,센터커피 서울숲점,
6,6,1460620031,인더매스 마장,서울 성동구 마장동 776-1,서울 성동구 마장로 270,NaN,카페,음식점 > 카페,http://place.map.kakao.com/1460620031,인더매스 마장,
7,7,547570065,옹근달,서울 성동구 성수동2가 333-52,서울 성동구 성수이로7길 41-1,02-499-5012,카페,음식점 > 카페,http://place.map.kakao.com/547570065,옹근달,
8,8,1842536906,멜로워 성수플래그쉽,서울 성동구 성수동2가 333-94,서울 성동구 성수이로7길 39,02-499-1112,카페,음식점 > 카페,http://place.map.kakao.com/1842536906,멜로워 성수플래그쉽,
9,9,1302549042,로우키,서울 성동구 성수동2가 302-16,서울 성동구 연무장3길 6,NaN,카페,음식점 > 카페,http://place.map.kakao.com/1302549042,로우키,


In [108]:
for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print(f'이번에 찾을 키워드 : {i}행 / 전체 {df.shape[0] -1}행중 {keyword}')
    if ' ' in keyword:
        join_keyword = '%20'.join(keyword.split())
    else:
        join_keyword = keyword
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={join_keyword}&sm=hty&style=v5"
        
        print(f'네이버 지도 url 정보: {naver_map_search_url}')

        driver.get(naver_map_search_url)
        time.sleep(2.5)
        df.iloc[i,-1] = driver.find_element(By.CSS_SELECTOR, '#ct > div.search_listview._content._ctList > ul > li > div.item_info > a.a_item.a_item_distance._linkSiteview').get_attribute('data-cid')
        
        time.sleep(1.0)
        
        #만약 검색 결과가 없다면?
    except Exception as e:
        print(e)
        pass
    
driver.quit()

이번에 찾을 키워드 : 0행 / 전체 674행중 블루보틀 성수 카페
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=블루보틀%20성수%20카페&sm=hty&style=v5
이번에 찾을 키워드 : 1행 / 전체 674행중 할아버지공장
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=할아버지공장&sm=hty&style=v5
이번에 찾을 키워드 : 2행 / 전체 674행중 쎈느
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=쎈느&sm=hty&style=v5
이번에 찾을 키워드 : 3행 / 전체 674행중 어니언컴퍼니
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=어니언컴퍼니&sm=hty&style=v5
이번에 찾을 키워드 : 4행 / 전체 674행중 대림창고
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=대림창고&sm=hty&style=v5
이번에 찾을 키워드 : 5행 / 전체 674행중 센터커피 서울숲점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=센터커피%20서울숲점&sm=hty&style=v5
이번에 찾을 키워드 : 6행 / 전체 674행중 인더매스 마장
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=인더매스%20마장&sm=hty&style=v5
이번에 찾을 키워드 : 7행 / 전체 674행중 옹근달
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=옹근달&sm=hty&style=v5
이번에 찾을 키워드

이번에 찾을 키워드 : 66행 / 전체 674행중 하루노유키
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=하루노유키&sm=hty&style=v5
이번에 찾을 키워드 : 67행 / 전체 674행중 마크69
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=마크69&sm=hty&style=v5
이번에 찾을 키워드 : 68행 / 전체 674행중 맛차차
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=맛차차&sm=hty&style=v5
이번에 찾을 키워드 : 69행 / 전체 674행중 아이오쓰리 성수사거리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=아이오쓰리%20성수사거리점&sm=hty&style=v5
이번에 찾을 키워드 : 70행 / 전체 674행중 브라운팟
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=브라운팟&sm=hty&style=v5
이번에 찾을 키워드 : 71행 / 전체 674행중 플디 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=플디%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 72행 / 전체 674행중 카페노티드 피치스
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20피치스&sm=hty&style=v5
이번에 찾을 키워드 : 73행 / 전체 674행중 천상가옥
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=천상가옥&sm=hty&style=v5
이번에 

이번에 찾을 키워드 : 131행 / 전체 674행중 플디 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=플디%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 132행 / 전체 674행중 카페노티드 피치스
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20피치스&sm=hty&style=v5
이번에 찾을 키워드 : 133행 / 전체 674행중 천상가옥
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=천상가옥&sm=hty&style=v5
이번에 찾을 키워드 : 134행 / 전체 674행중 비엔나커피하우스 서울숲역점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=비엔나커피하우스%20서울숲역점&sm=hty&style=v5
이번에 찾을 키워드 : 135행 / 전체 674행중 훔볼트
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=훔볼트&sm=hty&style=v5
이번에 찾을 키워드 : 136행 / 전체 674행중 업사이드커피 뚝섬점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=업사이드커피%20뚝섬점&sm=hty&style=v5
이번에 찾을 키워드 : 137행 / 전체 674행중 키커피컴퍼니 왕십리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=키커피컴퍼니%20왕십리점&sm=hty&style=v5
이번에 찾을 키워드 : 138행 / 전체 674행중 카페온화 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/

이번에 찾을 키워드 : 195행 / 전체 674행중 훔볼트
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=훔볼트&sm=hty&style=v5
이번에 찾을 키워드 : 196행 / 전체 674행중 업사이드커피 뚝섬점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=업사이드커피%20뚝섬점&sm=hty&style=v5
이번에 찾을 키워드 : 197행 / 전체 674행중 키커피컴퍼니 왕십리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=키커피컴퍼니%20왕십리점&sm=hty&style=v5
이번에 찾을 키워드 : 198행 / 전체 674행중 카페온화 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페온화%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 199행 / 전체 674행중 오우드
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=오우드&sm=hty&style=v5
이번에 찾을 키워드 : 200행 / 전체 674행중 카페노티드 성수
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20성수&sm=hty&style=v5
이번에 찾을 키워드 : 201행 / 전체 674행중 하루노유키
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=하루노유키&sm=hty&style=v5
이번에 찾을 키워드 : 202행 / 전체 674행중 마크69
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=마크69&

이번에 찾을 키워드 : 259행 / 전체 674행중 오우드
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=오우드&sm=hty&style=v5
이번에 찾을 키워드 : 260행 / 전체 674행중 카페노티드 성수
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20성수&sm=hty&style=v5
이번에 찾을 키워드 : 261행 / 전체 674행중 하루노유키
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=하루노유키&sm=hty&style=v5
이번에 찾을 키워드 : 262행 / 전체 674행중 마크69
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=마크69&sm=hty&style=v5
이번에 찾을 키워드 : 263행 / 전체 674행중 맛차차
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=맛차차&sm=hty&style=v5
이번에 찾을 키워드 : 264행 / 전체 674행중 아이오쓰리 성수사거리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=아이오쓰리%20성수사거리점&sm=hty&style=v5
이번에 찾을 키워드 : 265행 / 전체 674행중 브라운팟
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=브라운팟&sm=hty&style=v5
이번에 찾을 키워드 : 266행 / 전체 674행중 플디 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=플디%20성수점&sm=hty&style=v5


...

이번에 찾을 키워드 : 388행 / 전체 674행중 천상가옥
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=천상가옥&sm=hty&style=v5
이번에 찾을 키워드 : 389행 / 전체 674행중 비엔나커피하우스 서울숲역점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=비엔나커피하우스%20서울숲역점&sm=hty&style=v5
이번에 찾을 키워드 : 390행 / 전체 674행중 훔볼트
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=훔볼트&sm=hty&style=v5
이번에 찾을 키워드 : 391행 / 전체 674행중 업사이드커피 뚝섬점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=업사이드커피%20뚝섬점&sm=hty&style=v5
이번에 찾을 키워드 : 392행 / 전체 674행중 키커피컴퍼니 왕십리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=키커피컴퍼니%20왕십리점&sm=hty&style=v5
이번에 찾을 키워드 : 393행 / 전체 674행중 카페온화 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페온화%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 394행 / 전체 674행중 오우드
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=오우드&sm=hty&style=v5
이번에 찾을 키워드 : 395행 / 전체 674행중 카페노티드 성수
네이버 지도 url 정보: https://m.map.naver.com/search2/search.nav

이번에 찾을 키워드 : 452행 / 전체 674행중 키커피컴퍼니 왕십리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=키커피컴퍼니%20왕십리점&sm=hty&style=v5
이번에 찾을 키워드 : 453행 / 전체 674행중 카페온화 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페온화%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 454행 / 전체 674행중 오우드
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=오우드&sm=hty&style=v5
이번에 찾을 키워드 : 455행 / 전체 674행중 카페노티드 성수
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20성수&sm=hty&style=v5
이번에 찾을 키워드 : 456행 / 전체 674행중 하루노유키
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=하루노유키&sm=hty&style=v5
이번에 찾을 키워드 : 457행 / 전체 674행중 마크69
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=마크69&sm=hty&style=v5
이번에 찾을 키워드 : 458행 / 전체 674행중 맛차차
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=맛차차&sm=hty&style=v5
이번에 찾을 키워드 : 459행 / 전체 674행중 아이오쓰리 성수사거리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=아이오쓰리%20성수사

이번에 찾을 키워드 : 516행 / 전체 674행중 하루노유키
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=하루노유키&sm=hty&style=v5
이번에 찾을 키워드 : 517행 / 전체 674행중 마크69
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=마크69&sm=hty&style=v5
이번에 찾을 키워드 : 518행 / 전체 674행중 맛차차
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=맛차차&sm=hty&style=v5
이번에 찾을 키워드 : 519행 / 전체 674행중 아이오쓰리 성수사거리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=아이오쓰리%20성수사거리점&sm=hty&style=v5
이번에 찾을 키워드 : 520행 / 전체 674행중 브라운팟
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=브라운팟&sm=hty&style=v5
이번에 찾을 키워드 : 521행 / 전체 674행중 플디 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=플디%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 522행 / 전체 674행중 카페노티드 피치스
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20피치스&sm=hty&style=v5
이번에 찾을 키워드 : 523행 / 전체 674행중 천상가옥
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=천상가옥&sm=hty&style

이번에 찾을 키워드 : 581행 / 전체 674행중 플디 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=플디%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 582행 / 전체 674행중 카페노티드 피치스
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20피치스&sm=hty&style=v5
이번에 찾을 키워드 : 583행 / 전체 674행중 천상가옥
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=천상가옥&sm=hty&style=v5
이번에 찾을 키워드 : 584행 / 전체 674행중 비엔나커피하우스 서울숲역점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=비엔나커피하우스%20서울숲역점&sm=hty&style=v5
이번에 찾을 키워드 : 585행 / 전체 674행중 훔볼트
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=훔볼트&sm=hty&style=v5
이번에 찾을 키워드 : 586행 / 전체 674행중 업사이드커피 뚝섬점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=업사이드커피%20뚝섬점&sm=hty&style=v5
이번에 찾을 키워드 : 587행 / 전체 674행중 키커피컴퍼니 왕십리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=키커피컴퍼니%20왕십리점&sm=hty&style=v5
이번에 찾을 키워드 : 588행 / 전체 674행중 카페온화 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/

이번에 찾을 키워드 : 645행 / 전체 674행중 훔볼트
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=훔볼트&sm=hty&style=v5
이번에 찾을 키워드 : 646행 / 전체 674행중 업사이드커피 뚝섬점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=업사이드커피%20뚝섬점&sm=hty&style=v5
이번에 찾을 키워드 : 647행 / 전체 674행중 키커피컴퍼니 왕십리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=키커피컴퍼니%20왕십리점&sm=hty&style=v5
이번에 찾을 키워드 : 648행 / 전체 674행중 카페온화 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페온화%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 649행 / 전체 674행중 오우드
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=오우드&sm=hty&style=v5
이번에 찾을 키워드 : 650행 / 전체 674행중 카페노티드 성수
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20성수&sm=hty&style=v5
이번에 찾을 키워드 : 651행 / 전체 674행중 하루노유키
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=하루노유키&sm=hty&style=v5
이번에 찾을 키워드 : 652행 / 전체 674행중 마크69
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=마크69&

In [109]:
df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']

In [110]:
df['naver_map_url']

0      https://m.place.naver.com/restaurant/1725895520
1      https://m.place.naver.com/restaurant/1945220263
2        https://m.place.naver.com/restaurant/37879684
3      https://m.place.naver.com/restaurant/1925852247
4        https://m.place.naver.com/restaurant/37910590
                            ...                       
670    https://m.place.naver.com/restaurant/1165635472
671    https://m.place.naver.com/restaurant/1030413970
672    https://m.place.naver.com/restaurant/1022973489
673    https://m.place.naver.com/restaurant/1074358103
674    https://m.place.naver.com/restaurant/1320858854
Name: naver_map_url, Length: 675, dtype: object

In [111]:
df = df.loc[df['naver_map_url'] != 'https://m.place.naver.com/restaurant/']
df

,Unnamed: 0,id,place_name,address_name,road_address_name,phone,category_group_name,category_name,place_url,naver_keyword,naver_map_url
0,0,1492599844,블루보틀 성수 카페,서울 성동구 성수동1가 656-302,서울 성동구 아차산로 7,1533-6906,카페,음식점 > 카페,http://place.map.kakao.com/1492599844,블루보틀 성수 카페,https://m.place.naver.com/restaurant/1725895520
1,1,1301973155,할아버지공장,서울 성동구 성수동2가 309-133,서울 성동구 성수이로7가길 9,010-3934-2301,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/1301973155,할아버지공장,https://m.place.naver.com/restaurant/1945220263
2,2,1783691580,쎈느,서울 성동구 성수동2가 314-2,서울 성동구 연무장5길 20,02-469-2122,카페,음식점 > 카페,http://place.map.kakao.com/1783691580,쎈느,https://m.place.naver.com/restaurant/37879684
3,3,145791269,어니언컴퍼니,서울 성동구 성수동2가 277-135,서울 성동구 아차산로9길 8,02-1644-1941,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/145791269,어니언컴퍼니,https://m.place.naver.com/restaurant/1925852247
4,4,1128989913,대림창고,서울 성동구 성수동2가 322-32,서울 성동구 성수이로 78,02-499-9669,카페,음식점 > 카페 > 테마카페 > 갤러리카페,http://place.map.kakao.com/1128989913,대림창고,https://m.place.naver.com/restaurant/37910590
...,...,...,...,...,...,...,...,...,...,...,...
670,670,753723493,브라운팟,서울 성동구 행당동 286-19,서울 성동구 무학봉28길 16,02-2299-7796,카페,음식점 > 카페,http://place.map.kakao.com/753723493,브라운팟,https://m.place.naver.com/restaurant/1165635472
671,671,1431120740,플디 성수점,서울 성동구 성수동1가 13-156,서울 성동구 상원길 44,NaN,카페,음식점 > 카페,http://place.map.kakao.com/1431120740,플디 성수점,https://m.place.naver.com/restaurant/1030413970
672,672,1300323227,카페노티드 피치스,서울 성동구 성수동2가 301-16,서울 성동구 연무장3길 9,0507-1405-9377,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/1300323227,카페노티드 피치스,https://m.place.naver.com/restaurant/1022973489
673,673,1501931765,천상가옥,서울 성동구 성수동2가 322-4,서울 성동구 성수이로14길 14,NaN,카페,음식점 > 카페,http://place.map.kakao.com/1501931765,천상가옥,https://m.place.naver.com/restaurant/1074358103


In [112]:
df.to_csv('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe/성동구 카페 정보_url.csv')
df.to_json('/Users/dc/practice/recommendation/recomm_drf/data/crawl/cafe/성동구 카페 정보_url.json', orient='records', indent=4, force_ascii=False)

In [18]:
# 각 데이터들을 미리 리스트에 담은 다음, 마지막에 데이터 프레임에 합칠 것입니다.

# 네이버 지도의 유형 리스트
naver_map_type_list = []
# # 블로그 리뷰 리스트
# blog_review_list = []
# 블로그 리뷰 수 리스트
blog_review_qty_list = []
# 별점 점수(2022년 종료 예정) 정보 리스트
stars_list = []
# 별점 평가 수(2022년 종료 예정) 리스트
stars_qty_list = []
# 방문자 평가 참여자수 리스트
visitor_review_qty_list = []
# 방문자 리뷰 태그(호평) 리스트
visitor_review_list = ['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                       '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
                       '양이 많아요', '혼밥하기 좋아요']


df2 = df[['id', 'place_name', 'address_name', 'phone']]
# df2 = pd.DataFrame(df, columns=['id', 'place_name', 'address_name', 'phone'])



for i in visitor_review_list:
    df2[i] = ''

print(df2)
# print('==============')
# df2.iloc[:, 7]




            id     place_name       address_name           phone 커피가 맛있어요  \
0      8574728     토즈모임센터 종로점   서울 종로구 종로2가 84-8     02-736-0061            
1   1974266862       카페노티드 안국       서울 종로구 재동 86  0507-1337-9973            
2    853028118        어니언 안국점    서울 종로구 계동 146-1   070-7543-2123            
3    353512935     블루보틀 삼청 카페      서울 종로구 소격동 86       1533-6906            
4   1508329235           스태픽스    서울 종로구 필운동 32-1  0507-1341-2055            
5   1429224581          아키비스트   서울 종로구 효자동 164-7     02-738-1517            
6   1779571274          안녕인사동   서울 종로구 관훈동 155-2    02-6954-2910            
7      7912367           산모퉁이    서울 종로구 부암동 97-5     02-391-4737            
8     27187492        포비 광화문점     서울 종로구 청진동 246    02-2251-8125            
9      8082652         차마시는 뜰  서울 종로구 삼청동 35-169     02-722-7006            
10    24725284      테라로사 광화문점      서울 종로구 중학동 19             NaN            
11  1154772851        프릳츠 원서점   서울 종로구 원서동 221-1     02-747-8101            

/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_85985/1163794770.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[i] = ''
/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_85985/1163794770.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[i] = ''
/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_85985/1163794770.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [19]:
dict_review = {}
for i in visitor_review_list:
    dict_review[i] = []
    
print(dict_review)

# dict_review ={'커피가 맛있어요' : [], '빵이 맛있어요' : [], '친절해요' : []}

{'커피가 맛있어요': [], '디저트가 맛있어요': [], '빵이 맛있어요': [], '친절해요': [], '인테리어가 멋져요': [], '매장이 청결해요': [], '뷰가 좋아요': [], '음료가 맛있어요': [], '사진이 잘 나와요': [], '특별한 메뉴가 있어요': [], '대화하기 좋아요': [], '가성비가 좋아요': [], '집중하기 좋아요': [], '화장실이 깨끗해요': [], '주차하기 편해요': [], '좌석이 편해요': [], '음식이 맛있어요': [], '매장이 넓어요': [], '단체모임 하기 좋아요': [], '특별한 날 가기 좋아요': [], '재료가 신선해요': [], '양이 많아요': [], '혼밥하기 좋아요': []}


In [20]:
chromedriver = '/Users/dc/practice/recommendation/recomm_drf/cafe/chromedriver' 
#chromedriver = '/Users/dc/study_dir/recommendation/cafe/chromedriver'

# 메인 드라이버 : 별점 등을 크롤링
driver = webdriver.Chrome(chromedriver) 

# 서브 드라이버 : 리뷰 텍스트를 리뷰 탭 들어가서 크롤링
sub_driver = webdriver.Chrome(chromedriver)


/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_85985/1115631748.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)
/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_85985/1115631748.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  sub_driver = webdriver.Chrome(chromedriver)


In [21]:
# bs4 이용

for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):
    driver.get(url)
    sub_driver.get(url+"/review/visitor")
    time.sleep(3.0)
    
    html_driver = driver.page_source
    soup1 = BeautifulSoup(html_driver, 'html.parser')
    
    html_sub_driver = sub_driver.page_source
    soup2 = BeautifulSoup(html_sub_driver, 'html.parser')
    
    time.sleep(0.5)
    
    # 네이버 지도의 유형 분류
    try: 
        naver_map_type = soup1.select_one(".DJJvD").text
        print(f'naver_map_type: {naver_map_type}')
        naver_map_type_list.append(naver_map_type)    
    except:
        print('Element 요소 미존재')
        naver_map_type_list.append('null')
        
    time.sleep(0.5)
        
    # 블로그 리뷰 수
    try:
        blog_review_qty = soup1.select(".place_bluelink")[1].text
        print(f'블로그 리뷰 수: {blog_review_qty}')
        blog_review_qty_list.append(blog_review_qty)
    except:
        print('Element 요소 미존재')
        blog_review_qty_list.append('null')
        
    time.sleep(0.5)
        
    # 별점 점수(2022년 종료 예정)
    try:
        star_review_stars = soup1.select_one("div.dAsGb>span.PXMot.LXIwF>em").text 
        print(f'별점 점수(2022년 종료 예정): {star_review_stars}')
        stars_list.append(star_review_stars)
    except:
        print('Element 요소 미존재')
        stars_list.append('null')
        
    time.sleep(0.5)
        
    # 별점 평가 수(2022년 종료 예정)
    try:
        star_review_qty = soup2.select("div.Xj_yJ>span.m7jAR")[1]
        star_review_qty = star_review_qty.text
        print(f'별점 평가 수(2022년 종료 예정): {star_review_qty}')
        stars_qty_list.append(star_review_qty)
    except:
        print('Element 요소 미존재')
        stars_qty_list.append('null')
        
    time.sleep(0.5)
        
    # 방문자 평가 참여자수 + # 방문자 호평 부분
    try: # 방문자 평가 참여자수
        visitor_review_qty = soup2.select_one("div._Wmab>span.BPjKc").text
        visitor_review_qty = re.sub(r'[()]', '', visitor_review_qty)
        visitor_review_qty = re.sub(r'명 참여', '', visitor_review_qty)
        print(f'방문자 평가 참여자수: {visitor_review_qty}')
        visitor_review_qty_list.append(visitor_review_qty)
        
        # 방문자 호평 부분
        exist_review = []

        visitor_eval_list = soup2.select("div.k2tmh>ul.uNsI9>li.nbD78>div.CsBE9>span.nWiXa")
        visitor_choice_list = soup2.select("div.k2tmh>ul.uNsI9>li.nbD78>div.CsBE9>span.TwM9q")

        for i in range(5):
            visitor_eval = visitor_eval_list[i].text
            text_evaluation = re.sub('["]', '', visitor_eval)
            #print(f'방문자 호평 부분: {text_evaluation}')
            visitor_choice = visitor_choice_list[i].text
            visitor_choice = re.sub('이 키워드를 선택한 인원', '', visitor_choice)
            visitor_choice = visitor_choice.strip()
            #print(f'방문자 호평수: {visitor_choice}')
            if text_evaluation in visitor_review_list:
                dict_review[text_evaluation].append(visitor_choice)
                exist_review.append(text_evaluation)
                print(f'해당 플레이스 리뷰: {exist_review}')
        for review in visitor_review_list:
            if review not in exist_review:
                dict_review[review].append('')
        
    except:
        # 방문자 평가 참여자수
        print(f'방문자 평가 참여자수 10명 미만인 장소')
        visitor_review_qty = '9'
        visitor_review_qty_list.append(visitor_review_qty)
        # 방문자 호평 부분
        for i in visitor_review_list:
            dict_review[i].append('')
        
#     # 방문자 호평 부분
#     try:
#         exist_review = []

#         visitor_eval_list = soup2.select("div.k2tmh>ul.uNsI9>li.nbD78>div.CsBE9>span.nWiXa")
#         visitor_choice_list = soup2.select("div.k2tmh>ul.uNsI9>li.nbD78>div.CsBE9>span.TwM9q")

#         for i in range(5):
#             visitor_eval = visitor_eval_list[i].text
#             text_evaluation = re.sub('["]', '', visitor_eval)
#             #print(f'방문자 호평 부분: {text_evaluation}')
#             visitor_choice = visitor_choice_list[i].text
#             visitor_choice = re.sub('이 키워드를 선택한 인원', '', visitor_choice)
#             visitor_choice = visitor_choice.strip()
#             #print(f'방문자 호평수: {visitor_choice}')
#             if text_evaluation in visitor_review_list:
#                 dict_review[text_evaluation].append(visitor_choice)
#                 exist_review.append(text_evaluation)
#                 print(f'해당 플레이스 리뷰: {exist_review}')
#         for review in visitor_review_list:
#             if review not in exist_review:
#                 dict_review[review].append('')
#     except:
#         for i in visitor_review_list:
#             dict_review[i].append('')


/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_85985/3479362767.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):


  0%|          | 0/29 [00:00<?, ?it/s]

naver_map_type: 장소대여
블로그 리뷰 수: 블로그리뷰 33
별점 점수(2022년 종료 예정): 4.33
별점 평가 수(2022년 종료 예정): 108개 평점 99명 참여
방문자 평가 참여자수 10명 미만인 장소
naver_map_type: 카페
블로그 리뷰 수: 블로그리뷰 2,986
별점 점수(2022년 종료 예정): 4.44
별점 평가 수(2022년 종료 예정): 4,897개 평점 5,961명 참여
방문자 평가 참여자수: 3,868
해당 플레이스 리뷰: ['디저트가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '특별한 메뉴가 있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '특별한 메뉴가 있어요', '친절해요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '특별한 메뉴가 있어요', '친절해요', '매장이 청결해요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '특별한 메뉴가 있어요', '친절해요', '매장이 청결해요', '인테리어가 멋져요']
naver_map_type: 카페,디저트
블로그 리뷰 수: 블로그리뷰 3,537
별점 점수(2022년 종료 예정): 4.43
별점 평가 수(2022년 종료 예정): 4,057개 평점 5,648명 참여
방문자 평가 참여자수: 3,634
해당 플레이스 리뷰: ['디저트가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '인테리어가 멋져요', '커피가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '인테리어가 멋져요', '커피가 맛있어요', '친절해요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '인테리어가 멋져요', '커피가 맛있어요', '친절해요', '사진이 잘 나와요']
naver_map_type: 블루보틀
블로그 리뷰 수: 블로그리뷰 1,840
별점 점수(2022년 종료 예정): 4.44
별점 평가 수(2022년 종료 예정): 1,349개 평점 2,257명 참여
방문자 평가 참여

별점 점수(2022년 종료 예정): 4.38
별점 평가 수(2022년 종료 예정): 258개 평점 301명 참여
방문자 평가 참여자수: 12
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['커피가 맛있어요', '인테리어가 멋져요', '친절해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '인테리어가 멋져요', '친절해요', '매장이 청결해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '인테리어가 멋져요', '친절해요', '매장이 청결해요', '대화하기 좋아요']
naver_map_type: 차
블로그 리뷰 수: 블로그리뷰 878
별점 점수(2022년 종료 예정): 4.43
별점 평가 수(2022년 종료 예정): 1,025개 평점 1,343명 참여
방문자 평가 참여자수: 766
해당 플레이스 리뷰: ['특별한 메뉴가 있어요']
해당 플레이스 리뷰: ['특별한 메뉴가 있어요', '친절해요']
해당 플레이스 리뷰: ['특별한 메뉴가 있어요', '친절해요', '디저트가 맛있어요']
해당 플레이스 리뷰: ['특별한 메뉴가 있어요', '친절해요', '디저트가 맛있어요', '음료가 맛있어요']
naver_map_type: 이탈리아음식
블로그 리뷰 수: 블로그리뷰 508
Element 요소 미존재
Element 요소 미존재
방문자 평가 참여자수: 377
해당 플레이스 리뷰: ['음식이 맛있어요']
해당 플레이스 리뷰: ['음식이 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['음식이 맛있어요', '인테리어가 멋져요', '친절해요']
해당 플레이스 리뷰: ['음식이 맛있어요', '인테리어가 멋져요', '친절해요', '매장이 넓어요']
해당 플레이스 리뷰: ['음식이 맛있어요', '인테리어가 멋져요', '친절해요', '매장이 넓어요', '재료가 신선해요']
naver_map_type: 카페,디저트
블로그 리뷰 수: 블로그리뷰 784
별점 점수(2022년 종료 예정): 

In [22]:
driver.quit()
sub_driver.quit()

In [23]:
print(dict_review)

{'커피가 맛있어요': ['', '', '1399', '1324', '230', '1620', '', '144', '788', '', '513', '716', '250', '106', '266', '', '', '2', '245', '31', '86', '', '', '47', '151', '6', '', '', '130'], '디저트가 맛있어요': ['', '3106', '2236', '', '166', '437', '', '', '', '', '123', '459', '', '', '', '', '', '', '282', '233', '212', '', '', '', '312', '', '183', '', '99'], '빵이 맛있어요': ['', '', '', '', '', '', '', '', '1583', '', '', '', '861', '', '258', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '친절해요': ['', '873', '984', '540', '', '', '', '', '512', '219', '', '', '253', '31', '82', '175', '', '4', '', '57', '102', '148', '', '', '', '4', '256', '120', '59'], '인테리어가 멋져요': ['', '336', '1439', '435', '147', '357', '', '72', '', '164', '192', '291', '233', '36', '75', '332', '', '4', '301', '', '', '131', '', '34', '158', '5', '', '141', '71'], '매장이 청결해요': ['', '423', '', '431', '', '', '', '', '266', '', '123', '', '', '', '', '', '', '', '', '42', '', '111', '', '', '', '3', '', '', ''], '뷰가 좋아

In [24]:
print(len(dict_review['단체모임 하기 좋아요']))

29


In [25]:
df2['naver_map_type'] = naver_map_type_list            # 네이버 지도의 유형 리스트
df2['blog_review_qty_list'] = blog_review_qty_list     # 블로그 리뷰 수 리스트
df2['stars'] = stars_list                              # 별점 점수(2022년 종료 예정) 정보 리스트
df2['stars_qty'] = stars_qty_list                      # 별점 평가 수(2022년 종료 예정) 리스트
df2['visitor_review_qty'] = visitor_review_qty_list    # 방문자 평가 참여자수 리스트 


for key in dict_review:
    df2[key] = dict_review[key]

In [26]:
print(df2)

            id     place_name       address_name           phone 커피가 맛있어요  \
0      8574728     토즈모임센터 종로점   서울 종로구 종로2가 84-8     02-736-0061            
1   1974266862       카페노티드 안국       서울 종로구 재동 86  0507-1337-9973            
2    853028118        어니언 안국점    서울 종로구 계동 146-1   070-7543-2123     1399   
3    353512935     블루보틀 삼청 카페      서울 종로구 소격동 86       1533-6906     1324   
4   1508329235           스태픽스    서울 종로구 필운동 32-1  0507-1341-2055      230   
5   1429224581          아키비스트   서울 종로구 효자동 164-7     02-738-1517     1620   
6   1779571274          안녕인사동   서울 종로구 관훈동 155-2    02-6954-2910            
7      7912367           산모퉁이    서울 종로구 부암동 97-5     02-391-4737      144   
8     27187492        포비 광화문점     서울 종로구 청진동 246    02-2251-8125      788   
9      8082652         차마시는 뜰  서울 종로구 삼청동 35-169     02-722-7006            
10    24725284      테라로사 광화문점      서울 종로구 중학동 19             NaN      513   
11  1154772851        프릳츠 원서점   서울 종로구 원서동 221-1     02-747-8101      716   

In [27]:
df2.to_csv('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe/sample.csv')

In [28]:
df2_json = df2.to_json('/Users/dc/practice/recommendation/recomm_drf/data/crawl/cafe/review.json', orient='records', indent=4, force_ascii=False)
# print(df2_json)

In [39]:
# test(2022.10.25(화))
import os
# import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
import pandas as pd
import numpy as np
import json
import time
# from config.settings.base import CAFE, DATA_DIR, CSV_DATA_PATH
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from tqdm.notebook import tqdm
import re
from tqdm import *
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
df3 = pd.read_csv(os.path.join('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe', 'review_rating.csv'), sep=',')
df3 = df3.iloc[:, 1:4]
df3

,place_name,review,rating
0,토즈모임센터 종로점,커피가 맛있어요,NaN
1,토즈모임센터 종로점,디저트가 맛있어요,NaN
2,토즈모임센터 종로점,빵이 맛있어요,NaN
3,토즈모임센터 종로점,친절해요,NaN
4,토즈모임센터 종로점,인테리어가 멋져요,NaN
...,...,...,...
662,내자상회,단체모임 하기 좋아요,NaN
663,내자상회,특별한 날 가기 좋아요,NaN
664,내자상회,재료가 신선해요,NaN
665,내자상회,양이 많아요,NaN


In [42]:
x = df3.copy()
y = df3['review']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [43]:
rating_matrix = x_train.pivot(index='review', columns='place_name', values='rating')
rating_matrix

place_name,H라운지,개뿔,나무사이로,내자상회,대충유원지 인왕산,도트블랭킷,라카페갤러리,부빙,블루보틀 삼청 카페,산모퉁이,...,카페 하이웨이스트 익선점,카페노티드 안국,카페아르크,카페키이로,테라로사 광화문점,토즈모임센터 광화문점,토즈모임센터 종로점,포비 광화문점,포비브라이트 광화문점,프릳츠 원서점
review,,,,,,,,,,,,,,,,,,,,,
가성비가 좋아요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
단체모임 하기 좋아요,176.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
대화하기 좋아요,NaN,NaN,24.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
디저트가 맛있어요,NaN,NaN,NaN,99.0,NaN,NaN,NaN,233.0,NaN,NaN,...,312.0,3106.0,NaN,212.0,NaN,NaN,NaN,NaN,NaN,459.0
매장이 넓어요,324.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
매장이 청결해요,NaN,NaN,NaN,NaN,NaN,111.0,3.0,42.0,431.0,NaN,...,NaN,NaN,NaN,NaN,123.0,NaN,NaN,266.0,NaN,NaN
뷰가 좋아요,NaN,NaN,NaN,NaN,49.0,NaN,NaN,NaN,434.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,173.0
빵이 맛있어요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,258.0,NaN
사진이 잘 나와요,NaN,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,123.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
matrix_dummy = rating_matrix.copy().fillna(0)
review_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
review_similarity = pd.DataFrame(review_similarity,
                               index=rating_matrix.index,
                               columns=rating_matrix.index)

In [45]:
review_similarity

review,가성비가 좋아요,단체모임 하기 좋아요,대화하기 좋아요,디저트가 맛있어요,매장이 넓어요,매장이 청결해요,뷰가 좋아요,빵이 맛있어요,사진이 잘 나와요,양이 많아요,...,재료가 신선해요,좌석이 편해요,주차하기 편해요,집중하기 좋아요,친절해요,커피가 맛있어요,특별한 날 가기 좋아요,특별한 메뉴가 있어요,혼밥하기 좋아요,화장실이 깨끗해요
review,,,,,,,,,,,,,,,,,,,,,
가성비가 좋아요,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
단체모임 하기 좋아요,0.0,1.000000,0.000000,0.000000,0.965038,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
대화하기 좋아요,0.0,0.000000,1.000000,0.000000,0.000000,0.000696,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.020993,0.044090,0.0,0.000000,0.0,0.0
디저트가 맛있어요,0.0,0.000000,0.000000,1.000000,0.000000,0.004671,0.056399,0.000000,0.553061,0.0,...,0.000000,0.0,0.0,0.0,0.848669,0.121274,0.0,0.696248,0.0,0.0
매장이 넓어요,0.0,0.965038,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.064073,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
매장이 청결해요,0.0,0.000000,0.000696,0.004671,0.000000,1.000000,0.582220,0.000000,0.000000,0.0,...,0.201353,0.0,0.0,0.0,0.194786,0.608583,0.0,0.144287,0.0,0.0
뷰가 좋아요,0.0,0.000000,0.000000,0.056399,0.000000,0.582220,1.000000,0.000000,0.055337,0.0,...,0.000000,0.0,0.0,0.0,0.046988,0.536609,0.0,0.000000,0.0,0.0
빵이 맛있어요,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.178583,0.131460,0.0,0.204398,0.0,0.0
사진이 잘 나와요,0.0,0.000000,0.000000,0.553061,0.000000,0.000000,0.055337,0.000000,1.000000,0.0,...,0.000000,0.0,0.0,0.0,0.665041,0.011295,0.0,0.000000,0.0,0.0


In [46]:
#### 플레이스 전반에 대한 사용자 평가 경향을 고려한 함수 ####. axis=1 은 행 기준으로 평균을 구함(여기서 행은 review)
rating_mean = rating_matrix.mean(axis=1)
rating_mean

review
가성비가 좋아요               NaN
단체모임 하기 좋아요     176.000000
대화하기 좋아요         13.500000
디저트가 맛있어요       754.200000
매장이 넓어요         206.000000
매장이 청결해요        162.666667
뷰가 좋아요          234.000000
빵이 맛있어요         559.500000
사진이 잘 나와요       231.500000
양이 많아요                 NaN
음료가 맛있어요        168.857143
음식이 맛있어요        251.333333
인테리어가 멋져요       260.266667
재료가 신선해요        149.000000
좌석이 편해요                NaN
주차하기 편해요               NaN
집중하기 좋아요               NaN
친절해요            256.000000
커피가 맛있어요        414.928571
특별한 날 가기 좋아요           NaN
특별한 메뉴가 있어요     338.875000
혼밥하기 좋아요               NaN
화장실이 깨끗해요              NaN
dtype: float64

In [47]:
# 각 플레이스에 대한 리뷰 전체적인 평점
rating_matrix.T

review,가성비가 좋아요,단체모임 하기 좋아요,대화하기 좋아요,디저트가 맛있어요,매장이 넓어요,매장이 청결해요,뷰가 좋아요,빵이 맛있어요,사진이 잘 나와요,양이 많아요,...,재료가 신선해요,좌석이 편해요,주차하기 편해요,집중하기 좋아요,친절해요,커피가 맛있어요,특별한 날 가기 좋아요,특별한 메뉴가 있어요,혼밥하기 좋아요,화장실이 깨끗해요
place_name,,,,,,,,,,,,,,,,,,,,,
H라운지,NaN,176.0,NaN,NaN,324.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
개뿔,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
나무사이로,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,31.0,106.0,NaN,NaN,NaN,NaN
내자상회,NaN,NaN,NaN,99.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,59.0,130.0,NaN,NaN,NaN,NaN
대충유원지 인왕산,NaN,NaN,NaN,NaN,NaN,NaN,49.0,NaN,21.0,NaN,...,NaN,NaN,NaN,NaN,NaN,47.0,NaN,NaN,NaN,NaN
도트블랭킷,NaN,NaN,NaN,NaN,NaN,111.0,NaN,NaN,NaN,NaN,...,238.0,NaN,NaN,NaN,148.0,NaN,NaN,NaN,NaN,NaN
라카페갤러리,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,6.0,NaN,NaN,NaN,NaN
부빙,NaN,NaN,NaN,233.0,NaN,42.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,57.0,31.0,NaN,159.0,NaN,NaN
블루보틀 삼청 카페,NaN,NaN,NaN,NaN,NaN,431.0,434.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1324.0,NaN,NaN,NaN,NaN


In [48]:
# 각 플레이스 평점(rating_matrix.T)에 대한 리뷰별 경향(rating_mean. 리뷰의 평점평균)을 고려하여 플레이스 평점평균 '편차(bias)' 계산
rating_matrix.T - rating_mean

review,가성비가 좋아요,단체모임 하기 좋아요,대화하기 좋아요,디저트가 맛있어요,매장이 넓어요,매장이 청결해요,뷰가 좋아요,빵이 맛있어요,사진이 잘 나와요,양이 많아요,...,재료가 신선해요,좌석이 편해요,주차하기 편해요,집중하기 좋아요,친절해요,커피가 맛있어요,특별한 날 가기 좋아요,특별한 메뉴가 있어요,혼밥하기 좋아요,화장실이 깨끗해요
place_name,,,,,,,,,,,,,,,,,,,,,
H라운지,NaN,0.0,NaN,NaN,118.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
개뿔,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-252.0,NaN,NaN,NaN,NaN,NaN
나무사이로,NaN,NaN,10.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-225.0,-308.928571,NaN,NaN,NaN,NaN
내자상회,NaN,NaN,NaN,-655.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-197.0,-284.928571,NaN,NaN,NaN,NaN
대충유원지 인왕산,NaN,NaN,NaN,NaN,NaN,NaN,-185.0,NaN,-210.5,NaN,...,NaN,NaN,NaN,NaN,NaN,-367.928571,NaN,NaN,NaN,NaN
도트블랭킷,NaN,NaN,NaN,NaN,NaN,-51.666667,NaN,NaN,NaN,NaN,...,89.0,NaN,NaN,NaN,-108.0,NaN,NaN,NaN,NaN,NaN
라카페갤러리,NaN,NaN,-10.5,NaN,NaN,-159.666667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-252.0,-408.928571,NaN,NaN,NaN,NaN
부빙,NaN,NaN,NaN,-521.2,NaN,-120.666667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-199.0,-383.928571,NaN,-179.875,NaN,NaN
블루보틀 삼청 카페,NaN,NaN,NaN,NaN,NaN,268.333333,200.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,909.071429,NaN,NaN,NaN,NaN


In [49]:
rating_bias = (rating_matrix.T - rating_mean).T
rating_bias

place_name,H라운지,개뿔,나무사이로,내자상회,대충유원지 인왕산,도트블랭킷,라카페갤러리,부빙,블루보틀 삼청 카페,산모퉁이,...,카페 하이웨이스트 익선점,카페노티드 안국,카페아르크,카페키이로,테라로사 광화문점,토즈모임센터 광화문점,토즈모임센터 종로점,포비 광화문점,포비브라이트 광화문점,프릳츠 원서점
review,,,,,,,,,,,,,,,,,,,,,
가성비가 좋아요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
단체모임 하기 좋아요,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
대화하기 좋아요,NaN,NaN,10.500000,NaN,NaN,NaN,-10.500000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
디저트가 맛있어요,NaN,NaN,NaN,-655.200000,NaN,NaN,NaN,-521.200000,NaN,NaN,...,-442.200000,2351.800,NaN,-542.200000,NaN,NaN,NaN,NaN,NaN,-295.200000
매장이 넓어요,118.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
매장이 청결해요,NaN,NaN,NaN,NaN,NaN,-51.666667,-159.666667,-120.666667,268.333333,NaN,...,NaN,NaN,NaN,NaN,-39.666667,NaN,NaN,103.333333,NaN,NaN
뷰가 좋아요,NaN,NaN,NaN,NaN,-185.000000,NaN,NaN,NaN,200.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-61.000000
빵이 맛있어요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-301.500000,NaN
사진이 잘 나와요,NaN,NaN,NaN,NaN,-210.500000,NaN,NaN,NaN,NaN,-108.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [51]:
# 모델별 RMSE를 계산하는 함수(test set 활용하여 계산)
def score(model, neigbor_size=0):
    # 테스트 데이터의 review와 place_name 간 pair를 맞춰 튜플형 원소 리스트 데이터를 만듬
    id_pairs = zip(x_test['review'], x_test['place_name'])
    # 모든 review-플레이스 짝에 대해서 주어진 예측모델에 의해 예측값 계산 및 리스트형 데이터 생성
    y_pred = np.array([model(review, place, neigbor_size) for (review, place) in id_pairs])
    # 실제 평점값
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

In [34]:
# id_pairs_list = list(zip(x_test['review'], x_test['place_name']))
# print(id_pairs_list)

[('특별한 날 가기 좋아요', '카페키이로'), ('디저트가 맛있어요', '스태픽스'), ('단체모임 하기 좋아요', '내자상회'), ('주차하기 편해요', '어니언 안국점'), ('단체모임 하기 좋아요', '서울서 둘째로잘하는집'), ('친절해요', '내자상회'), ('음료가 맛있어요', '토즈모임센터 종로점'), ('가성비가 좋아요', '개뿔'), ('화장실이 깨끗해요', '카페 레이어드 안국점'), ('재료가 신선해요', '테라로사 광화문점'), ('친절해요', '산모퉁이'), ('재료가 신선해요', '라카페갤러리'), ('디저트가 맛있어요', '대충유원지 인왕산'), ('대화하기 좋아요', '카페노티드 안국'), ('뷰가 좋아요', '개뿔'), ('화장실이 깨끗해요', '카페노티드 안국'), ('집중하기 좋아요', '차마시는 뜰'), ('인테리어가 멋져요', '프릳츠 원서점'), ('화장실이 깨끗해요', '토즈모임센터 광화문점'), ('뷰가 좋아요', '도트블랭킷'), ('특별한 메뉴가 있어요', '개뿔'), ('사진이 잘 나와요', '산모퉁이'), ('친절해요', '슬로우파크'), ('디저트가 맛있어요', '슬로우파크'), ('특별한 날 가기 좋아요', '라카페갤러리'), ('양이 많아요', '토즈모임센터 광화문점'), ('주차하기 편해요', '카페키이로'), ('화장실이 깨끗해요', '대충유원지 인왕산'), ('음료가 맛있어요', 'H라운지'), ('집중하기 좋아요', '카페아르크'), ('특별한 메뉴가 있어요', '카페 하이웨이스트 익선점'), ('혼밥하기 좋아요', '개뿔'), ('커피가 맛있어요', '어니언 안국점'), ('음식이 맛있어요', '안녕인사동'), ('빵이 맛있어요', '청수당'), ('집중하기 좋아요', '카페노티드 안국'), ('음료가 맛있어요', '도트블랭킷'), ('빵이 맛있어요', '아키비스트'), ('주차하기 편해요', '서울서 둘째로잘하는집'), ('매장이 넓어요', '아키비스트'), ('음식이 맛있어요', '카페노티

In [52]:
# rating_matrix > 0 을 통해 나온 True 값들을 numpy array로 만들고, 이렇게 나온 True, False 구분값들을 1과 0의 binary 값으로 변환
rating_binary_1 = np.array(rating_matrix > 0).astype(float)
rating_binary_2 = rating_binary_1.T

counts = np.dot(rating_binary_1, rating_binary_2)
counts = pd.DataFrame(counts,
                      index=rating_matrix.index,
                      columns=rating_matrix.index).fillna(0)

In [53]:
# 각 review 평가가 되어있는 플레이스의 갯수를 나타내기 (대각선 = 각 review 평가가 되어있는 플레이스의 갯수, 다른 값들은 review간 공통으로 평가되어있는 플레이스의 갯수)
counts

review,가성비가 좋아요,단체모임 하기 좋아요,대화하기 좋아요,디저트가 맛있어요,매장이 넓어요,매장이 청결해요,뷰가 좋아요,빵이 맛있어요,사진이 잘 나와요,양이 많아요,...,재료가 신선해요,좌석이 편해요,주차하기 편해요,집중하기 좋아요,친절해요,커피가 맛있어요,특별한 날 가기 좋아요,특별한 메뉴가 있어요,혼밥하기 좋아요,화장실이 깨끗해요
review,,,,,,,,,,,,,,,,,,,,,
가성비가 좋아요,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
단체모임 하기 좋아요,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
대화하기 좋아요,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
디저트가 맛있어요,0.0,0.0,0.0,10.0,0.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,5.0,6.0,0.0,5.0,0.0,0.0
매장이 넓어요,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
매장이 청결해요,0.0,0.0,1.0,1.0,0.0,6.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,4.0,4.0,0.0,2.0,0.0,0.0
뷰가 좋아요,0.0,0.0,0.0,2.0,0.0,1.0,5.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0
빵이 맛있어요,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,0.0
사진이 잘 나와요,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0


In [86]:
SIG_LEVEL = 2
MIN_RATINGS = 1

In [87]:
def CF_knn_bias_sig(review, place, neighbor_size=0):
    if place in rating_bias.columns:
        # input으로 들어온 review와 다른 review 간의 유사도 추출
        sim_scores = review_similarity[review].copy()
        # 해당 플레이스에 대한 review 평가 경향고려한(rating_bias 데이터프레임) 평점 정보 추출
        place_ratings = rating_bias[place].copy()
        # 플레이스 평가에 등록되어있지 않은 review인지 추출
        no_rating = place_ratings.isnull() # True, False
        # review가 공통으로 등록되어있는 플레이스의 갯수를 할당
        common_counts = counts[review]
        
        low_significance = common_counts < SIG_LEVEL   # SIG_LEVEL = 미리 설정한 숫자 보다 작은 review는 True로 변환
        # 플레이스 평가등록되지 않은 review or 지정한 SIG_LEVEL보다 적게 공통의 플레이스를 평가한 review의 index 추출
        none_rating_idx = place_ratings[no_rating | low_significance].index

        place_ratings = place_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)

        if neighbor_size == 0:
            # review 평가 경향을 반영한 가중평균 (플레이스점수*가중치(review간 유사도) / 가중치의 합)
            prediction = np.dot(sim_scores, place_ratings) / sim_scores.sum()
            # '가중평균(위의 prediction)' + '플레이스 전반에 대한 해당 review 평균 평점' 하여 reveiw 카테고리의 최종 플레이스 예상 평점 구함 
            prediction += rating_mean[review]
        else: 
            if len(sim_scores) > MIN_RATINGS:     # MIN_RATINGS = 미리 설정한 최소 review보다 현재 플레이스를 평가한 유사한 review가 큰 경우만
                print('=============='*5)
                print('MIN_RATINGS 보다 큰 상황')
                print('=============='*5)
                neighbor_size = min(neighbor_size, len(sim_scores))
                sim_scores = np.array(sim_scores)
                place_ratings = np.array(place_ratings)
                # 유사도 오름차순 정렬후 user index 추출
                review_idx = np.argsort(sim_scores)
                # 입력한 neighbor_size 만큼 높은 유사도 정보 추출
                sim_scores = sim_scores[review_idx][-neighbor_size:]
                # 평점 정보 추출
                place_ratings = place_ratings[review_idx][-neighbor_size:]
                # review 평가 경향을 반영한 가중평균 (플레이스점수*가중치(review간 유사도) / 가중치의 합)
                prediction = np.dot(sim_scores, place_ratings) / sim_scores.sum()
                # '가중평균(위의 prediction)' + '플레이스 전반에 대한 해당 review 평균 평점' 하여 reveiw 카테고리의 최종 플레이스 예상 평점 구함 
                prediction += rating_mean[review]
            else:
                print('=============='*5)
                print('MIN_RATINGS 보다 작은 상황')
                print('=============='*5)
                # 원래 review의 평균 평점을 예상값으로 지정
                prediction = rating_mean[review]
    else:   # place에 대항하는 플레이스 정보가 rating_bias에 없는 경우
        # 원래 review의 평균 평점을 예상값으로 지정
        prediction = rating_mean[review]  # 강의 원본

    return prediction

In [59]:
CF_knn_bias_sig('커피가 맛있어요', 'H라운지', 3)

414.92857142857144

In [63]:
# SIG_LEVEL = 3
# MIN_RATINGS = 3   # 1이상
score(CF_knn_bias_sig, neigbor_size=4)

nan

In [75]:
# 원본
def recom_place(review, n_items, neighbor_size=0):
    # review가 등록되어 있는 플레이스 정보 추출
    review_place = rating_matrix.loc[review].copy()
    print(f'review_place: {review_place}')
    print('=============='*5)
    
    for place in rating_matrix.columns:
        # 해당 플레이스에 대한 review가 이미 존재하는 경우, 추천 리스트에서 제외???(장소 추천시스템에선 불필요...?)
        if pd.notnull(review_place.loc[place]):
            review_place.loc[place] = 0
        else:
            review_place.loc[place] = CF_knn_bias_sig(review, place, neighbor_size)
            
    place_sort = review_place.sort_values(ascending=False)[:n_items]
    print(f'place_sort: {place_sort}')
    recom_places = movies.loc[place_sort.index]
    recommendations = recom_places['title']
    return recommendations

In [88]:
# 테스트 중
def recom_place(review, n_items, neighbor_size=0):
    # review가 등록되어 있는 플레이스 정보 추출
    review_place = rating_matrix.loc[review].copy()
    print(f'review_place: {review_place}')
    print('=============='*5)
    
    for place in rating_matrix.columns:
        review_place.loc[place] = CF_knn_bias_sig(review, place, neighbor_size)
        
    print(f'review_place 최종: {review_place}')
            
    place_sort = review_place.sort_values(ascending=False)[:n_items]
    print(f'place_sort: {place_sort}')
#     recom_places = movies.loc[place_sort.index]
#     recommendations = recom_places['title']
#     return recommendations

In [89]:
recom_place('매장이 넓어요', 3, 3)

review_place: place_name
H라운지             324.0
개뿔                 NaN
나무사이로              NaN
내자상회               NaN
대충유원지 인왕산          NaN
도트블랭킷              NaN
라카페갤러리             NaN
부빙                 NaN
블루보틀 삼청 카페         NaN
산모퉁이               NaN
서울서 둘째로잘하는집        NaN
스태픽스               NaN
슬로우파크             88.0
아키비스트              NaN
안녕인사동              NaN
어니언 안국점            NaN
차마시는 뜰             NaN
청수당                NaN
카페 레이어드 안국점        NaN
카페 하이웨이스트 익선점      NaN
카페노티드 안국           NaN
카페아르크              NaN
카페키이로              NaN
테라로사 광화문점          NaN
토즈모임센터 광화문점        NaN
토즈모임센터 종로점         NaN
포비 광화문점            NaN
포비브라이트 광화문점        NaN
프릳츠 원서점            NaN
Name: 매장이 넓어요, dtype: float64
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RAT

In [7]:
# ratings = df3.groupby(['place_name'])[['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
#             a                           '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
#                                        '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
#                                        '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
#                                        '양이 많아요', '혼밥하기 좋아요']].sum()

# ratings
ratings = df3.pivot(index=['place_name'],
         columns=['place_name'],
         values=['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                 '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                 '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                 '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
                 '양이 많아요', '혼밥하기 좋아요'])

ratings

ValueError: The name place_name occurs multiple times, use a level number

In [161]:
df3 = df2[['place_name', '커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                       '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요'
                       '양이 많아요', '혼밥하기 좋아요']]

In [162]:
place_rating = df3.copy()
place_rating = place_rating.replace('', np.nan)
place_rating = place_rating.fillna(0)
# place_rating = place_rating.T
print(place_rating)
place_rating.to_csv('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe/sample2.csv')

       place_name 커피가 맛있어요 디저트가 맛있어요 빵이 맛있어요 친절해요 인테리어가 멋져요 매장이 청결해요 뷰가 좋아요  \
0      토즈모임센터 종로점        0         0       0    0         0        0      0   
1        카페노티드 안국        0      3102       0  871       333      423      0   
2         어니언 안국점     1396      2231       0  983      1436        0      0   
3      블루보틀 삼청 카페     1323         0       0  536       434      428    430   
4            스태픽스      230       166       0    0       148        0    323   
5           아키비스트     1618       437       0    0       356        0      0   
6           안녕인사동        0         0       0    0         0        0      0   
7            산모퉁이      144         0       0    0        72        0    324   
8         포비 광화문점      784         0    1580  509         0      265      0   
9          차마시는 뜰        0         0       0  219       164        0    192   
10      테라로사 광화문점      513       123       0    0       192      124      0   
11        프릳츠 원서점      712       455       0    0   

In [139]:
# rating = place_rating.groupby(['place_name'])[['커피가 맛있어요']].sum()
# rating = place_rating.groupby(['place_name'])[['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
#                        '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
#                        '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
#                        '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요'
#                        '양이 많아요', '혼밥하기 좋아요']].sum()


# rating

In [163]:
print(place_rating.columns[1:])
review_list = place_rating.columns[1:]
print(len(review_list))

Index(['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요',
       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요',
       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', '음식이 맛있어요',
       '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요양이 많아요', '혼밥하기 좋아요'],
      dtype='object')
22


In [164]:
rating_matrix = place_rating.pivot_table(index=['place_name'], 
                                   columns=['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                                       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                                       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                                       '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
                                       '양이 많아요', '혼밥하기 좋아요'], 
                                   values=['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                                       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                                       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                                       '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
                                       '양이 많아요', '혼밥하기 좋아요'],
                                  aggfunc='sum')


In [165]:
rating_matrix

place_name
H라운지
개뿔
나무사이로
내자상회
대충유원지 인왕산
도트블랭킷
라카페갤러리
부빙
블루보틀 삼청 카페
산모퉁이


In [159]:
rating_matrix = rating_matrix.stack()

ValueError: Must pass non-zero number of levels/codes

In [150]:
rating_matrix

place_name
H라운지
개뿔
나무사이로
내자상회
대충유원지 인왕산
도트블랭킷
라카페갤러리
부빙
블루보틀 삼청 카페
산모퉁이


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [86]:
x = place_rating.copy()
y = place_rating.iloc[0].to_list()
print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

['토즈모임센터 종로점', '카페노티드 안국', '어니언 안국점', '블루보틀 삼청 카페', '스태픽스', '아키비스트', '안녕인사동', '산모퉁이', '포비 광화문점', '차마시는 뜰', '테라로사 광화문점', '프릳츠 원서점', '카페 레이어드 안국점', '나무사이로', '포비브라이트 광화문점', 'H라운지', '토즈모임센터 광화문점', '개뿔', '청수당', '부빙', '카페키이로', '도트블랭킷', '카페아르크', '대충유원지 인왕산', '카페 하이웨이스트 익선점', '라카페갤러리', '서울서 둘째로잘하는집', '슬로우파크', '내자상회']


In [ ]:
# 셀레니움 이용

for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):
    driver.get(url)
    sub_driver.get(url+"/review/visitor")
    time.sleep(3.0)
#     driver.implicitly_wait(10)
#     sub_driver.implicitly_wait(10)

    # driver
    # 네이버 지도의 유형 분류
    try:
        naver_map_type = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#_title > span.DJJvD"))
        )
    except NoSuchElementException:
        print('Element 요소 미존재')
        naver_map_type_list.append('null')
    else:
        driver.implicitly_wait(5)
        print(f'naver_map_type: {naver_map_type.text}')
        naver_map_type_list.append(naver_map_type.text)
        driver.implicitly_wait(5)
    
#     # 네이버 지도의 유형 분류
#     naver_map_type = driver.find_element(By.CSS_SELECTOR, "#_title > span.DJJvD").text
#     print(f'naver_map_type: {naver_map_type}')
#     naver_map_type_list.append(naver_map_type)
#     #time.sleep(2.0)
#     driver.implicitly_wait(5)

    # 블로그 리뷰 수
    try:
#         blog_review_qty = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(3) > a > em"))
#         )
        blog_review_qty = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(3) > a > em").text
        if not blog_review_qty:
            print('blog_review_qty ELEMNT does not exist & try other ELEMENT')
            driver.implicitly_wait(5)
#             blog_review_qty = WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(2) > a > em"))
#             )
            blog_review_qty = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(2) > a > em").text
    except NoSuchElementException:
        print('Element 요소 미존재')
        blog_review_qty_list.append('null')
    else:
        driver.implicitly_wait(5)
        print(f'블로그 리뷰 수: {blog_review_qty}')
        blog_review_qty_list.append(blog_review_qty)
        #time.sleep(2.0)
        driver.implicitly_wait(5)

    # 별점 점수(2022년 종료 예정)
    try:
#         star_review_stars = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span.PXMot.LXIwF > em"))
#         )
        star_review_stars = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span.PXMot.LXIwF > em").text
        driver.implicitly_wait(5)
        print(f'별점 점수(2022년 종료 예정): {star_review_stars}')
        stars_list.append(star_review_stars)
        #time.sleep(2.0)
    except NoSuchElementException:
        print('Element 요소 미존재')
        stars_list.append('null')



    # sub_driver
    
    # 별점 평가 수(2022년 종료 예정)  //*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/span[2]
    try:
#         star_review_qty = WebDriverWait(sub_driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, r'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[3]/span[2]'))
#         )
        star_review_qty = sub_driver.find_element(By.XPATH, r'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[3]/span[2]').text
        if not star_review_qty:
            print('star_review_qty ELEMENT does not exist & try other ELEMENT')
            sub_driver.implicitly_wait(5)
#             star_review_qty = WebDriverWait(sub_driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, r'//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/span[2]'))
#             )          
            star_review_qty = sub_driver.find_element(By.XPATH, r'//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/span[2]').text
    except NoSuchElementException:
        print('Element 요소 미존재')
        stars_qty_list.append('null')
    else:
        sub_driver.implicitly_wait(5)
        print(f'별점 평가 수(2022년 종료 예정): {star_review_qty}')
        stars_qty_list.append(star_review_qty)
        #time.sleep(2.0)


    # 방문자 평가 참여자수
    try:
        visitor_review_qty_element = WebDriverWait(sub_driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.mdJ86 > div > div > div._Wmab > span"))
        )
        visitor_review_qty = visitor_review_qty_element.text
        #visitor_review_qty = sub_driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.mdJ86 > div > div > div._Wmab > span").text
        visitor_review_qty = re.sub('[\(\)]', '', visitor_review_qty)
        visitor_review_qty = re.sub('명 참여', '', visitor_review_qty)
        sub_driver.implicitly_wait(5)
        print(f'방문자 평가 참여자수: {visitor_review_qty}')
        visitor_review_qty_list.append(visitor_review_qty)
        #time.sleep(2.0)
    except:
        print(f'방문자 평가 참여자수 10명 미만인 장소')
        visitor_review_qty = '9'
        visitor_review_qty_list.append(visitor_review_qty)


    # 방문자 호평 부분   # k2tmh   ,   uNsI9
    # visitor_eval_list = sub_driver.find_elements(By.CLASS_NAME, "k2tmh")
    # print(f'방문자 호평 부분: {visitor_eval_list}')

    try:
        visitor_eval_list_element = WebDriverWait(sub_driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.mdJ86 > div > div > div.k2tmh > ul"))
        )
        visitor_eval_list = visitor_eval_list_element.text
        #visitor_eval_list = sub_driver.find_elements(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.mdJ86 > div > div > div.k2tmh > ul")
        sub_driver.implicitly_wait(5)
        print(f'방문자 호평 부분: {visitor_eval_list}')
        #sub_driver.implicitly_wait(5)
        
        for visitor_eval in visitor_eval_list[:5]:
            print('===========================')
            # 평가 전체 내역
            evaluation = visitor_eval.text
            print(f'evaluation: {evaluation}')
            print('===========================')
            print('for문을 통해 평가 내역 추출')
            evaluation = evaluation.split('\n')
            
            exist_review = []
            
            for i in range(0, 14, 3):
                print(evaluation[i])
                text_evaluation = re.sub('["]', '', evaluation[i])
                print('========'*5)
                print(text_evaluation)

                choose_evaluation = evaluation[i+2]
                print(f'choose_evaluation: {choose_evaluation}')
                print('========'*5)
                
                if text_evaluation in visitor_review_list:
                    dict_review[text_evaluation].append(choose_evaluation)
                    exist_review.append(text_evaluation)

#                 if text_evaluation in visitor_review_list:
#                     dict_review[text_evaluation].append(choose_evaluation)
            for review in visitor_review_list:
                if review not in exist_review:
                    dict_review[review].append('')
        #time.sleep(3.0)
    except:
        for i in visitor_review_list:
            dict_review[i].append('')


#     naver_map_type_list.append(naver_map_type)
#     blog_review_qty_list.append(blog_review_qty)
#     stars_list.append(star_review_stars)
#     stars_qty_list.append(star_review_qty)
#     visitor_review_qty_list.append(visitor_review_qty)

#     # 리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
#     except Exception as e:
#         print(f"{i}행 문제가 발생: {e}")
        
#         # 내역이 없으므로 null을 임시로 넣어줍니다.
#         naver_map_type_list.append('null')
#         blog_review_qty_list.append('null')
#         stars_list.append('null')
#         stars_qty_list.append('null')
#         visitor_review_qty_list.append('null')
        
driver.quit()
sub_driver.quit()

In [ ]:
print(dict_review)
print(len(dict_review['빵이 맛있어요']))

In [ ]:
print(naver_map_type_list)
print(len(naver_map_type_list))

In [ ]:
print(visitor_review_qty_list)
print(len(visitor_review_qty_list))

In [ ]:
df2['naver_map_type'] = naver_map_type_list            # 네이버 지도의 유형 리스트
df2['blog_review_qty_list'] = blog_review_qty_list     # 블로그 리뷰 수 리스트
df2['stars'] = stars_list                              # 별점 점수(2022년 종료 예정) 정보 리스트
df2['stars_qty'] = stars_qty_list                      # 별점 평가 수(2022년 종료 예정) 리스트
df2['visitor_review_qty'] = visitor_review_qty_list    # 방문자 평가 참여자수 리스트 


for key in dict_review:
    df2[key] = dict_review[key]

In [ ]:
print(stars_qty_list)

In [ ]:
df['naver_map_url'] = ''
df

In [ ]:
df.iloc[0,-1]